In [1]:
!pip install -U scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/d0/0b/26ad95cf0b747be967b15fb71a06f5ac67aba0fd2f9cd174de6edefc4674/scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 52.4 MB/s eta 0:00:0000:010:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [3]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

In [4]:
df = pd.read_csv("/kaggle/input/case2data/pricerunner_aggregate.csv")
df.head()

,Product ID,Product Title,Merchant ID,Cluster ID,Cluster Label,Category ID,Category Label
0,1,apple iphone 8 plus 64gb silver,1,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
1,2,apple iphone 8 plus 64 gb spacegrau,2,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
2,3,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,3,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
3,4,apple iphone 8 plus 64gb space grey,4,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones
4,5,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,5,1,Apple iPhone 8 Plus 64GB,2612,Mobile Phones


In [5]:
df[" Category Label"].value_counts()

 Category Label
Fridge Freezers     5501
Mobile Phones       4081
Washing Machines    4044
CPUs                3862
Fridges             3584
TVs                 3564
Dishwashers         3424
Digital Cameras     2697
Microwaves          2342
Freezers            2212
Name: count, dtype: int64

In [6]:
df[" Cluster Label"].value_counts()

 Cluster Label
Canon IXUS 185              27
Canon PowerShot SX730 HS    24
Samsung UE49NU7100          24
Apple iPhone 8 Plus 64GB    23
Samsung UE65NU7100          23
                            ..
Panasonic TX-65FX600B        1
Samsung HG55ED890WB          1
Samsung UE55MU6192           1
Panasonic TX-24D302B         1
Neff K4316                   1
Name: count, Length: 12849, dtype: int64

In [7]:
df["first_word"] = df[" Cluster Label"].str.split().str[0]
df["first_word"].nunique()

284

In [8]:
X = df["first_word"]
y = df[" Category ID"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

le = LabelEncoder()

X_train = le.fit_transform(X_train)

In [9]:
le.classes_

array(['AEG', 'AMD', 'ASUS', 'Acer', 'Adelberg', 'AgfaPhoto', 'Akai',
       'Alcatel', 'Aligator', 'Allcall', 'Allview', 'Amazon', 'Amica',
       'Amplicomms', 'Apple', 'Archos', 'Ariston', 'Audioline', 'Avtex',
       'Bauknecht', 'Baumatic', 'Bea-fon', 'Beafon', 'Beko', 'Belling',
       'Benq', 'Binatone', 'Blackberry', 'Blackview', 'Blaupunkt',
       'Blizzard', 'Blomberg', 'Blu', 'Bluboo', 'Bomann', 'Bosch', 'Bq',
       'Breville', 'Britannia', 'Bush', 'CASO', 'CAT', 'CDA', 'Candy',
       'Canon', 'Caple', 'Casio', 'Cello', 'Changhong', 'Cisco',
       'Clatronic', 'Constructa', 'Cookology', 'Cookworks', 'Crosscall',
       'Cubot', 'Cylinda', 'Cyrus', 'Daewoo', 'De', 'DeLonghi', 'Dell',
       'Denver', 'Dewalt', 'DigiHome', 'Digiquest', 'Disney', 'Dometic',
       'Doogee', 'Doro', 'Dyon', 'Dörr', 'E98CWW', 'Easypix', 'Ebac',
       'Elari', 'ElectrIQ', 'Electra', 'Electrolux', 'Elephone',
       'Emporia', 'Energizer', 'Energy', 'Essentials', 'Estar',
       'Exquisit', 'F

In [14]:
X_test = ["AMD" if x not in le.classes_ else x for x in X_test]

X_test = le.transform(X_test)

In [15]:
forest = RandomForestClassifier(n_estimators=10, max_depth=3, random_state=42)

forest.fit(pd.DataFrame(X_train), y_train)

forest.score(pd.DataFrame(X_test), y_test)

0.2790598895653405

In [16]:
print(classification_report(y_test, forest.predict(pd.DataFrame(X_test))))

              precision    recall  f1-score   support

        2612       0.00      0.00      0.00       818
        2614       0.21      0.91      0.34       723
        2615       1.00      0.54      0.70       771
        2617       0.51      0.03      0.06       542
        2618       0.00      0.00      0.00       469
        2619       0.00      0.00      0.00       662
        2620       0.41      0.33      0.36       796
        2621       0.00      0.00      0.00       439
        2622       0.22      0.55      0.31      1115
        2623       0.00      0.00      0.00       728

    accuracy                           0.28      7063
   macro avg       0.23      0.24      0.18      7063
weighted avg       0.25      0.28      0.21      7063



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Catboost


In [17]:
X = df[[" Merchant ID", " Cluster ID"]]
y = df[" Category ID"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

cat = CatBoostClassifier(
    iterations=10,
    learning_rate=0.1,
    random_seed=42,
    logging_level="Silent",
    task_type="GPU",
    # cat_features=[" Merchant ID", " Cluster ID"],
)

cat.fit(X_train, y_train)

In [18]:
pred = cat.predict(X_test)

print(classification_report(y_test, pred))

              precision    recall  f1-score   support

        2612       0.98      1.00      0.99       818
        2614       0.97      0.98      0.98       723
        2615       0.98      0.97      0.98       771
        2617       1.00      0.93      0.97       542
        2618       0.95      1.00      0.97       469
        2619       1.00      0.96      0.98       662
        2620       0.97      0.99      0.98       796
        2621       0.98      0.98      0.98       439
        2622       0.99      1.00      0.99      1115
        2623       0.99      1.00      1.00       728

    accuracy                           0.98      7063
   macro avg       0.98      0.98      0.98      7063
weighted avg       0.98      0.98      0.98      7063



In [19]:
X = df['Product Title']
y = df[' Category ID']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = make_pipeline(TfidfVectorizer(), SVC(kernel='linear'))

pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('svc', SVC(kernel='linear'))])

In [20]:
# Предсказание на тестовых данных
pred = pipeline.predict(X_test)

# Вывод отчета о классификации
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

        2612       0.98      1.00      0.99       818
        2614       0.98      0.99      0.98       723
        2615       1.00      1.00      1.00       771
        2617       1.00      1.00      1.00       542
        2618       1.00      0.96      0.97       469
        2619       0.90      0.96      0.93       662
        2620       0.94      0.96      0.95       796
        2621       0.98      0.93      0.95       439
        2622       0.97      0.93      0.95      1115
        2623       0.91      0.92      0.92       728

    accuracy                           0.96      7063
   macro avg       0.96      0.96      0.96      7063
weighted avg       0.96      0.96      0.96      7063

